In [ ]:
import os
from dotenv import load_dotenv 
load_dotenv()

from langchain_groq import ChatGroq
os.environ['GROQ_API_KEY'] = os.getenv('GROQ_API_KEY')
llm = ChatGroq(model='llama-3.1-8b-instant')
result=llm.invoke("Hello")
result

In [ ]:
from typing_extensions import Literal
from pydantic import BaseModel,Field
from langchain_core.messages import HumanMessage,SystemMessage
from typing_extensions import TypedDict

# Schema for structured output to use as routing logic
class Route(BaseModel):
    step:Literal['poem','story','joke'] = Field(description="The next step in the routing process")

## Argument the LLM with schema for structured output
router=llm.with_structured_output(Route)

#state Schema
class State(TypedDict):
    input:str
    decision:str
    output:str

#nodes 
def Story(state:State):
    """Write a story"""

    print("Story node is called")
    result = llm.invoke(state['input'])
    return {'output':result.content}

def Joke(state:State):
    """Write a joke"""

    print("Joke node is called")
    result = llm.invoke(state['input'])
    return {"output":result.content}

def Poem(state:State):
    """Write a poem"""

    print("Poem node is called")
    result = llm.invoke(state['input'])
    return {"output":result.content}
    
def llm_call_router(state:State):
    """Route the input to the appropiate node"""

    decision = router.invoke(
        [
            SystemMessage(content="Route the input to the story,joke or poem based on the users request"),
            HumanMessage(content=state['input'])
        ]
    )
    return {'decision':decision.step}

## Conditional Edge function to route to the appropiate node
def route_decision(state:State):
    #Return the node name you want to visit
    if state['decision'] == 'story':
        return "Story"
    elif state['decision'] == "joke":
        return "Joke"
    elif state['decision'] == 'poem':
        return "Poem"
    
from langgraph.graph import StateGraph,START,END
from IPython.display import display,Image
#build workflow
router_builder= StateGraph(State)

#Add node
router_builder.add_node("Story",Story)
router_builder.add_node("Joke",Joke)
router_builder.add_node("Poem",Poem)
router_builder.add_node('llm_call_router',llm_call_router)

#Add edges to connect nodes
router_builder.add_edge(START,'llm_call_router')
router_builder.add_conditional_edges('llm_call_router',
                                     route_decision,{"Story": "Story",
                                                    "Joke": "Joke",
                                                    "Poem": "Poem",
    },)


router_builder.add_edge("Story", END)
router_builder.add_edge("Joke", END)
router_builder.add_edge("Poem", END)

# Compile workflow
router_workflow = router_builder.compile()

# Show the workflow
display(Image(router_workflow.get_graph().draw_mermaid_png()))



In [ ]:
state=router_workflow.invoke({"input":"Write me a Joke about Rahul Gandhi in hindi language with very funny way"})
print(state["output"])